In [1]:
import pandas as pd
import yfinance as yf

## Prepare data
### loading data

In [2]:
df = pd.read_csv('/Users/bowenduan/Applications/OneDrive/200_Knowledge/210_Academe/211_University/PACE University/105_Spring 2021/CS668 Analytics Capstone/CS668-2021/data/AAPL.csv')
df.head()

,Date,Open,High,Low,Close,Volume
0,2011-02-09,10.926702,11.043910,10.916859,11.018068,482745200
1,2011-02-10,10.994382,11.074673,10.705516,10.906707,928550000
2,2011-02-11,10.913166,11.006992,10.875942,10.977768,367572800
3,2011-02-14,10.975924,11.058675,10.973463,11.049447,310416400
4,2011-02-15,11.049754,11.073750,10.999304,11.071596,284174800


In [3]:
# get the previouse day close to label the data
df['NxtClose'] = df['Close'].shift(-1)
df.head()

,Date,Open,High,Low,Close,Volume,NxtClose
0,2011-02-09,10.926702,11.043910,10.916859,11.018068,482745200,10.906707
1,2011-02-10,10.994382,11.074673,10.705516,10.906707,928550000,10.977768
2,2011-02-11,10.913166,11.006992,10.875942,10.977768,367572800,11.049447
3,2011-02-14,10.975924,11.058675,10.973463,11.049447,310416400,11.071596
4,2011-02-15,11.049754,11.073750,10.999304,11.071596,284174800,11.170958


In [4]:
# label the observation 
# 1 for price going up, 0 for price going down
# calculate the daily percent change
df['Ptc'] = round(df['NxtClose']/df['Close'] - 1, 4)*100
df['sgn'] = [1 if  x > 0 else 0 for x in df['Ptc']]
df.head()

,Date,Open,High,Low,Close,Volume,NxtClose,Ptc,sgn
0,2011-02-09,10.926702,11.043910,10.916859,11.018068,482745200,10.906707,-1.01,0
1,2011-02-10,10.994382,11.074673,10.705516,10.906707,928550000,10.977768,0.65,1
2,2011-02-11,10.913166,11.006992,10.875942,10.977768,367572800,11.049447,0.65,1
3,2011-02-14,10.975924,11.058675,10.973463,11.049447,310416400,11.071596,0.20,1
4,2011-02-15,11.049754,11.073750,10.999304,11.071596,284174800,11.170958,0.90,1


### calculate variables

In [5]:
# calculate ratios as machine learning variables 
# these ratio calculation will be done by ta-lib
from talib import abstract

# moving average 
ma_list = [5, 10, 20, 30, 60]
for i in ma_list:
    df[f'ma{i}'] = pd.Series(abstract.SMA(df, timeperiod = i,price = 'Close'), index = df.index)

# volumne moving average     
ma_vol_list = [5, 10, 20, 30, 60]
for i in ma_vol_list:
    df[f'ma_vol{i}'] = pd.Series(abstract.SMA(df, timeperiod = i, price = 'Volume'), index = df.index)

df.tail()

,Date,Open,High,Low,Close,Volume,NxtClose,Ptc,sgn,ma5,ma10,ma20,ma30,ma60,ma_vol5,ma_vol10,ma_vol20,ma_vol30,ma_vol60
2511,2021-02-02,135.527476,136.106612,134.409152,134.788589,83305400,133.740158,-0.78,0,135.845007,137.223946,133.172000,132.618494,126.467268,130038140.0,124494920.0,1.142834e+08,1.165266e+08,1.084359e+08
2512,2021-02-03,135.557435,135.567429,133.410649,133.740158,89880900,137.184998,2.58,1,134.223431,137.414662,133.318282,132.860799,126.718809,119845560.0,123051060.0,1.138942e+08,1.131046e+08,1.078274e+08
2513,2021-02-04,136.096628,137.194977,134.389172,137.184998,84183100,136.759995,-0.31,0,134.283340,137.466585,133.856977,133.165677,127.030011,108157960.0,119416420.0,1.103490e+08,1.118690e+08,1.073228e+08
2514,2021-02-05,137.350006,137.419998,135.860001,136.759995,75524000,136.910004,0.11,1,135.282718,137.256334,134.158744,133.334902,127.373570,87826640.0,115578100.0,1.086463e+08,1.087563e+08,1.060063e+08
2515,2021-02-08,136.029999,136.949997,134.929993,136.910004,68005611,NaN,NaN,0,135.876749,136.676659,134.411596,133.539749,127.725454,80179802.2,106617491.1,1.067983e+08,1.080824e+08,1.048394e+08


### clean data

In [6]:
# reset index to datatime
# delet Next day close (future value)
# delet Ptc (future vale)
df = df.set_index(['Date'])
df = df.drop(['NxtClose','Ptc'], axis= 1)

In [7]:
df.tail()

,Open,High,Low,Close,Volume,sgn,ma5,ma10,ma20,ma30,ma60,ma_vol5,ma_vol10,ma_vol20,ma_vol30,ma_vol60
Date,,,,,,,,,,,,,,,,
2021-02-02,135.527476,136.106612,134.409152,134.788589,83305400,0,135.845007,137.223946,133.172000,132.618494,126.467268,130038140.0,124494920.0,1.142834e+08,1.165266e+08,1.084359e+08
2021-02-03,135.557435,135.567429,133.410649,133.740158,89880900,1,134.223431,137.414662,133.318282,132.860799,126.718809,119845560.0,123051060.0,1.138942e+08,1.131046e+08,1.078274e+08
2021-02-04,136.096628,137.194977,134.389172,137.184998,84183100,0,134.283340,137.466585,133.856977,133.165677,127.030011,108157960.0,119416420.0,1.103490e+08,1.118690e+08,1.073228e+08
2021-02-05,137.350006,137.419998,135.860001,136.759995,75524000,1,135.282718,137.256334,134.158744,133.334902,127.373570,87826640.0,115578100.0,1.086463e+08,1.087563e+08,1.060063e+08
2021-02-08,136.029999,136.949997,134.929993,136.910004,68005611,0,135.876749,136.676659,134.411596,133.539749,127.725454,80179802.2,106617491.1,1.067983e+08,1.080824e+08,1.048394e+08


## prepare training data

In [8]:
df = df.dropna(axis=0)

In [9]:
# training period: 20130228~20180228
df_train = df[df.index < '2018-02-08' ]
# resample the training data 
sgn_1 = df_train[df_train['sgn'] == 1]
sgn_0 = df_train[df_train['sgn'] == 0]
len_1 = len(sgn_1)
len_0 = len(sgn_0)
print(f'Up record has {len_1} row, while the down records have {len_0} row \n')


Up record has 878 row, while the down records have 824 row 



In [10]:
from sklearn.utils import resample
#resample
df_majority = sgn_1
df_minority = sgn_0

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= len_1,    # to match majority class
                                 random_state=42) # reproducible results
# Combine majority class with upsampled minority class
df_train = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
print('after resample, the records situation is: \n',df_train.sgn.value_counts())
df_train_x = df_train.drop(['sgn'], axis=1)
df_train_y = df_train['sgn']
print(f'{len(df_train_x)} row training data is prepated.')


after resample, the records situation is: 
 1    878
0    878
Name: sgn, dtype: int64
1756 row training data is prepated.


## Prapre test data 

In [11]:
# testing period: 20180228~20190228
df_test = df[(df.index > '2018-02-08') & (df.index < '2020-02-08')]
df_test_x = df_test.drop(['sgn'], axis=1)
df_test_y = df_test['sgn']
print(f'{len(df_test_x)} row test data is prepared. \n')

502 row test data is prepared. 



## train model

In [12]:
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load

clf = RandomForestClassifier(max_depth=2, random_state=42)
clf_model = clf.fit(df_train_x,df_train_y)
dump(clf_model, 'AAPL_model.joblib')
df_test_predict =  clf_model.predict(df_test_x)
#save prediction results
df_test['predict'] = df_test_predict  
df_test.to_csv('AAPL_test_predict.csv')

In [13]:
# test model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print(f'classification reports \n')
print(confusion_matrix(df_test_y, df_test_predict))
print(f'Accuracy rate is {accuracy_score(df_test_y, df_test_predict)}')

classification reports 

[[225   0]
 [277   0]]
Accuracy rate is 0.448207171314741


## predition on new data and assign singal

In [14]:
# paper trading period: 20200208 ~ 20210208
# print('Starting Prediction on trading date')
df_trade = df[df.index > '2020-02-08'].drop('sgn', axis=1)
# load model
model = load('AAPL_model.joblib')
df_trade['predict'] = model.predict(df_trade)
# save prediction file 
df_trade.to_csv('AAPL_prediction_sgn.csv')

## back testing